# Part I

In [ ]:
# Install libraries for local development
# !conda install --yes cvxopt numpy matplotlib pandas scipy

# Import libraries

from cvxopt import matrix, solvers
from datetime import datetime
from google.colab import drive
from itertools import combinations
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import randint
from scipy.spatial.distance import cdist
import time

In [ ]:
# Mount Google Drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
# Data preparation

def load_data(path):
  data = np.loadtxt(path)
  xs = data[:,1:]
  ys = np.ndarray.flatten(data[:,:1]).astype(int)
  return xs, ys

# Path for local development
# path = "/users/danielmay/Coursework/COMP0086/Coursework 2"

path = "/content/drive/My Drive/Colab Notebooks/sl_coursework2"

data_xs, data_ys = load_data(f"{path}/data/zipcombo.dat")

In [ ]:
# Data exploration

# Display examples
examples_to_show = list(range(17,18))

for example in examples_to_show:
  image = np.reshape(data_xs[example], (16, 16))

  plt.imshow(image, cmap='gray')
  plt.axis('off')
  plt.title(f"Label: {data_ys[example]}\n")
  plt.savefig(f"example_{example}", dpi=300)
  plt.show()

# Display an example of each class
fig, ax = plt.subplots(2,5)
ax = ax.flatten()
for i in range(10):
    example = np.argwhere(data_ys == i)[0]
    image = np.reshape(data_xs[example], (16, 16))
    ax[i].imshow(image, cmap='gray')
    ax[i].axis('off')
fig.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout()
plt.show()
fig.savefig('all_classes', dpi=300)

# Plot class counts
counts = np.bincount(data_ys)
print("Counts:", counts)
plt.bar(range(len(counts)), counts)
plt.xticks(range(len(counts)))
plt.xlabel('Class')
plt.ylabel('Number of examples')
plt.title('Number of examples per class in the data set\n')
plt.savefig('examples_per_class', dpi=300)
plt.show()

In [ ]:
# Data processing functions

def split_data(xs, ys, frac=0.8):
  """
  Randomly split data into a training set and test set
  """
  n = xs.shape[0]
  shuffled_indices = [*range(n)]
  np.random.shuffle(shuffled_indices)
  # Generate random training and test sets
  training_n = int(frac * n)
  training_indices, test_indices = shuffled_indices[:training_n], shuffled_indices[training_n:]
  train_xs, test_xs = xs[training_indices], xs[test_indices]
  train_ys, test_ys = ys[training_indices], ys[test_indices]

  return train_xs, train_ys, test_xs, test_ys

def generate_validation_folds(xs, ys, k):
  """
  Randomly split data into k folds, returning a list of the form:
  [train_xs, train_ys, validation_xs, validation_ys]
  """
  result = []

  n = xs.shape[0]
  shuffled_indices = np.arange(n)
  np.random.shuffle(shuffled_indices)
  # Split the shuffled indices into k chunks for the validation sets
  for validation_indices in np.array_split(shuffled_indices, k):
      train_indices = np.setdiff1d(shuffled_indices, validation_indices)
      # Extract training and validation sets
      train_xs, validation_xs = xs[train_indices], xs[validation_indices]
      train_ys, validation_ys = ys[train_indices], ys[validation_indices]
      result.append([train_xs, train_ys, validation_xs, validation_ys])
  return result

def normalize_confusion_matrix(confusion_matrix):
  """
  Normalize a confusion matrix of misclassifications
  """
  return np.nan_to_num(confusion_matrix / confusion_matrix.sum(axis=1)[:,np.newaxis])

# Kernel functions

def polynomial_kernel(X, Y, p):
  """Compute polynomial kernel matrix, K, of x and y with exponent p"""
  return np.dot(X, Y.T) ** p

def gaussian_kernel(P, Q, c):
  """Compute Gaussian kernel matrix, K, of P and Q with width c"""
  return np.exp(-c * cdist(P, Q) ** 2)

In [ ]:
MIN_EPOCHS = 10
MAX_EPOCHS = 50

class MultiClassPerceptronOvR:
  """
  One-versus-rest (OvR) kernel perceptron
  """

  def __init__(self, num_classes):
    self.num_classes = num_classes
    self.num_classifiers = num_classes

    # Initialised by set_{polynomial/gaussian}_kernel method
    self.kernel_function = None

    # Initialised by train method
    self.train_xs = None
    self.alpha = None
  
  def set_polynomial_kernel(self, degree):
    self.kernel_function = lambda x, y: polynomial_kernel(x, y, degree)
  
  def set_gaussian_kernel(self, width):
    self.kernel_function = lambda p, q: gaussian_kernel(p, q, width)

  def train(self, train_xs, train_ys):

    # Initialisation
    num_examples = train_xs.shape[0]
    self.train_xs = train_xs
    self.alpha = np.zeros((self.num_classifiers, num_examples))

    # Compute Gram matrix, K
    gram_matrix = self.kernel_function(train_xs, train_xs)

    # Training loop
    running_mistakes = 0
    train_accuracy_list = []

    converged = False
    epoch = 0

    while (converged == False) and (epoch <= MAX_EPOCHS):

      running_mistakes = 0

      # Shuffle the training data
      shuffled_indices = [*range(num_examples)]
      np.random.shuffle(shuffled_indices)

      for example in shuffled_indices:
        x = train_xs[example]
        y = train_ys[example]

        # Predict (highest confidence)
        confidences = np.dot(self.alpha, gram_matrix[example, ])
        y_hat = np.argmax(confidences)

        # Update (if prediction is incorrect)
        if (y_hat != y):
          running_mistakes += 1
        
          self.alpha[y, example] += 1 # Raise score of right answer
          self.alpha[y_hat, example] -= 1 # Lower score of wrong answer

      # Calculate training accuracy
      train_accuracy = (num_examples - running_mistakes) / float(num_examples)
      train_accuracy_list.append(train_accuracy)

      # Early stopping
      if epoch >= MIN_EPOCHS:
        if (np.mean(train_accuracy_list[-5:]) - np.mean(train_accuracy_list[-10:-5])) < 0.01:
          converged = True

          # Count number of non-zero alpha vector elements per class
          print(f"Number of non-zero alpha vector elements per class: {np.count_nonzero(self.alpha, axis=1)}")
      
      epoch += 1
                
    return train_accuracy_list

  def test(self, test_xs, test_ys):

    num_examples = test_xs.shape[0] 

    # Compute kernel matrix, K
    K = self.kernel_function(self.train_xs, test_xs)

    running_mistakes = 0
    misclassifications_by_example = np.zeros((num_examples))
    confusion_matrix = np.zeros((self.num_classes, self.num_classes))

    # Make a prediction for each example
    for example in range(num_examples):
      y = test_ys[example]
      y_hat = np.argmax(np.dot(self.alpha, K[:, example]))

      # Track mistakes
      if (y_hat != y):
        running_mistakes += 1
        misclassifications_by_example[example] += 1
        confusion_matrix[y][y_hat] += 1
    
    # Calculate test accuracy
    test_accuracy = (num_examples - running_mistakes) / float(num_examples)
    
    return test_accuracy, confusion_matrix, misclassifications_by_example

## 1. Basic results

In [ ]:
NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

DEGREES = range(1, 8)
RUNS = 20

summary_data = []

for degree in DEGREES:

  train_accuracy_list = np.empty((RUNS))
  test_accuracy_list = np.empty((RUNS))

  for run in range(RUNS):

    # Generate train/test split
    train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

    # Initialise model
    model = MultiClassPerceptronOvR(NUM_CLASSES)
    model.set_polynomial_kernel(degree)

    # Train model and evaluate on test set
    train_accuracy = model.train(train_xs, train_ys)[-1]
    test_accuracy, _, _ = model.test(test_xs, test_ys)

    train_accuracy_list[run] = train_accuracy
    test_accuracy_list[run] = test_accuracy

    print(f"Polynomial degree: {degree}. Run: {run}. Train accuracy: {train_accuracy}. Test accuracy: {test_accuracy}.")

  # Calculate mean and std for train and test accuracy
  train_error_mean, train_error_std = 1 - np.mean(train_accuracy_list), np.std(train_accuracy_list)
  test_error_mean, test_error_std = 1 - np.mean(test_accuracy_list), np.std(test_accuracy_list)

  summary_data.append([degree, train_error_mean, train_error_std, test_error_mean, test_error_std])
  
# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('degree', 'train_error_mean', 'train_error_std', 'test_error_mean', 'test_error_std'))

# Save summary as CSV
csv_filename = f"Q1_1_{datetime.now().strftime('%Y_%m_%d_%H_%M')}.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

## 2. Cross-validation & 3. Confusion matrix

In [ ]:
NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

DEGREES = range(1, 8)
RUNS = 20

FOLDS = 5

summary_data = []
confusion_matrices = np.empty((RUNS, NUM_CLASSES, NUM_CLASSES))
misclassifications_by_example_acc = np.zeros((data_xs.shape[0]))

for run in range(RUNS):

  mean_val_accuracy_list = np.empty((len(DEGREES)))

  # Generate train/test split
  train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

  for i, degree in enumerate(DEGREES):

    # Generate validation folds
    validation_folds = generate_validation_folds(train_xs, train_ys, k=5)

    train_accuracy_list = np.empty((FOLDS))
    val_accuracy_list = np.empty((FOLDS))

    for fold, (fold_train_xs, fold_train_ys, fold_validation_xs, fold_validation_ys) in enumerate(validation_folds):

      # Initialise model
      model = MultiClassPerceptronOvR(NUM_CLASSES)
      model.set_polynomial_kernel(degree=degree)

      # Train model and evaluate on validation set
      train_accuracy = model.train(fold_train_xs, fold_train_ys)[-1]
      val_accuracy, _, _ = model.test(fold_validation_xs, fold_validation_ys)

      train_accuracy_list[fold] = train_accuracy
      val_accuracy_list[fold] = val_accuracy

    print(f"Polynomial degree: {degree}. Run: {run}. Train accuracy list: {train_accuracy_list}. Val accuracy: {val_accuracy_list}.")

    mean_val_accuracy_list[i] = np.mean(val_accuracy_list)

  # Get the degree with maximum mean validation accuracy
  argmax = np.argmax(mean_val_accuracy_list)
  degree_star = DEGREES[argmax]
  
  # Initialise model
  model = MultiClassPerceptronOvR(NUM_CLASSES)
  model.set_polynomial_kernel(degree=degree)

  # Train model
  train_accuracy = model.train(train_xs, train_ys)[-1]
  train_error = 1 - train_accuracy

  # Evaluate on test set
  test_accuracy, confusion_matrix, _ = model.test(test_xs, test_ys)
  test_error = 1 - test_accuracy

  # Evaluate on entire data set to help discover the five hardest to predict data items
  data_accuracy, _, misclassifications_by_example = model.test(data_xs, data_ys)
  misclassifications_by_example_acc += misclassifications_by_example

  confusion_matrices[run] = normalize_confusion_matrix(confusion_matrix)
  print(confusion_matrices[run])

  summary_data.append([degree_star, train_error, test_error])
  print(summary_data)

# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('degree', 'train_error', 'test_error'))

# Create DataFrames of the confusion matrix mean and std
confusion_matrix_mean = pd.DataFrame(np.mean(confusion_matrices, axis=0))
confusion_matrix_std = pd.DataFrame(np.std(confusion_matrices, axis=0))

# Save summary as CSV
csv_filename = f"Q1_2_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
summary.to_csv(f"{ path}/{csv_filename}", index=False)

# Save confusion matrix mean and std as CSV
csv_filename = f"Q1_3_mean_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
confusion_matrix_mean.to_csv(f"{path}/{csv_filename}", float_format='%.3g')
 
csv_filename = f"Q1_3_std_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
confusion_matrix_std.to_csv(f"{path}/{csv_filename}", float_format='%.3g')

summary

## 4. Five hardest to predict data items

In [ ]:
# The most misclassified indices & number of times they were misclassified (during RUNS runs)

most_misclassified_indices = np.argsort(misclassifications_by_example_acc)[::-1]
misclassified_at_least_once = np.count_nonzero(misclassifications_by_example_acc)

n_most_misclassified_indices = most_misclassified_indices[:10]
n_most_misclassified_counts = misclassifications_by_example_acc[n_most_misclassified_indices]

print("Data set size:", len(data_ys))

print(f"Number of distinct examples misclassified over {RUNS} runs: {misclassified_at_least_once} ")

print("Most misclassified indices:", n_most_misclassified_indices)
print("Misclassification counts:", n_most_misclassified_counts)

# Plot the five most frequently misclassified examples
for i, example in enumerate(most_misclassified_indices[:5]):
  image = np.reshape(data_xs[example], (16, 16))

  example_misclassifications = int(misclassifications_by_example_acc[example])

  plt.imshow(image, cmap='gray')
  plt.axis('off')
  plt.title(f"Label: {data_ys[example]}. Misclassified {example_misclassifications} times.\n")
  plt.savefig(f"Q1_4_{i}_most", dpi=300)
  plt.show()

## 5. Repeat 1. and 2. with Gaussian kernel

### 5.a. Basic exploration

In [ ]:
NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

WIDTHS = [2 ** k for k in range(-15, 10)]
RUNS = 20

summary_data = []

for width in WIDTHS:

  train_accuracy_list = np.empty((RUNS))
  test_accuracy_list = np.empty((RUNS))

  for run in range(RUNS):

    # Generate train/test split
    train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

    # Initialise model
    model = MultiClassPerceptronOvR(NUM_CLASSES)
    model.set_gaussian_kernel(width)

    # Train model and evaluate on test set
    train_accuracy = model.train(train_xs, train_ys)[-1]
    test_accuracy, _, _ = model.test(test_xs, test_ys)

    train_accuracy_list[run] = train_accuracy
    test_accuracy_list[run] = test_accuracy

    print(f"Gaussian kernel width: {width}. Run: {run}. Train accuracy: {train_accuracy}. Test accuracy: {test_accuracy}.")

  # Calculate mean and std for train and test accuracy
  train_error_mean, train_error_std = 1 - np.mean(train_accuracy_list), np.std(train_accuracy_list)
  test_error_mean, test_error_std = 1 - np.mean(test_accuracy_list), np.std(test_accuracy_list)

  summary_data.append([width, train_error_mean, train_error_std, test_error_mean, test_error_std])
  
# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('width', 'train_error_mean', 'train_error_std', 'test_error_mean', 'test_error_std'))

# Save summary as CSV
csv_filename = f"Q1_5_a_{datetime.now().strftime('%Y_%m_%d_%H_%M')}.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

### 5.b. Cross-validation

In [ ]:
NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

WIDTHS = [2 ** k for k in range(-10, -3)]
RUNS = 20

FOLDS = 5

summary_data = []

for run in range(RUNS):

  mean_val_accuracy_list = np.empty((len(WIDTHS)))

  # Generate train/test split
  train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

  for i, width in enumerate(WIDTHS):

    # Generate validation folds
    validation_folds = generate_validation_folds(train_xs, train_ys, k=5)

    train_accuracy_list = np.empty((FOLDS))
    val_accuracy_list = np.empty((FOLDS))

    for fold, (fold_train_xs, fold_train_ys, fold_validation_xs, fold_validation_ys) in enumerate(validation_folds):

      # Initialise model
      model = MultiClassPerceptronOvR(NUM_CLASSES)
      model.set_gaussian_kernel(width=width)

      # Train model and evaluate on validation set
      train_accuracy = model.train(fold_train_xs, fold_train_ys)[-1]
      val_accuracy, _, _ = model.test(fold_validation_xs, fold_validation_ys)

      train_accuracy_list[fold] = train_accuracy
      val_accuracy_list[fold] = val_accuracy

    print(f"Gaussian kernel width: {width}. Run: {run}. Train accuracy list: {train_accuracy_list}. Val accuracy: {val_accuracy_list}.")

    mean_val_accuracy_list[i] = np.mean(val_accuracy_list)

  # Get the width with maximum mean validation accuracy
  argmax = np.argmax(mean_val_accuracy_list)
  width_star = WIDTHS[argmax]
  
  # Initialise model
  model = MultiClassPerceptronOvR(NUM_CLASSES)
  model.set_gaussian_kernel(width=width)

  # Train model and evaluate on test set
  train_accuracy = model.train(train_xs, train_ys)[-1]
  test_accuracy, _, _ = model.test(test_xs, test_ys)

  train_error = 1 - train_accuracy
  test_error = 1 - test_accuracy

  summary_data.append([width_star, train_error, test_error])

# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('width', 'train_error', 'test_error'))

# Save summary as CSV
csv_filename = f"Q1_5_b_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

## 6. Repeat 1. and 2. with alternative method


In [ ]:
MIN_EPOCHS = 2
MAX_EPOCHS = 50

class MultiClassPerceptronOvO:
  """
  One-versus-one (OvO) kernel perceptron
  """

  def __init__(self, num_classes):
    self.num_classes = num_classes
    self.num_classifiers = int(num_classes * (num_classes - 1) / 2)

    self.class_combinations = list(combinations(range(num_classes), 2))
    self.class_combinations_to_alpha_index = dict(zip(self.class_combinations, range(self.num_classifiers)))

    # Initialised by set_polynomial_kernel method
    self.kernel_function = None

    # Initialised by train method
    self.train_xs = None
    self.alpha = None

  def __get_alpha_index(self, i, j):
    """
    Get the row index for two classes i, j of a classifier in the alpha matrix
    """
    key = tuple(sorted([i, j]))
    return self.class_combinations_to_alpha_index[key]
  
  def set_polynomial_kernel(self, degree):
    self.kernel_function = lambda x, y: polynomial_kernel(x, y, degree)

  def __predict(self, K_matrix, example):
    """
    Predict the class of an example as the one with the most votes over all
    classifiers
    """
    confidences = np.dot(self.alpha, K_matrix[:, example])
    binary_predictions = np.sign(confidences)
    predicted_class_indices = np.clip(binary_predictions, a_min=0, a_max=None).astype(int)

    multiclass_predictions = np.empty((self.num_classifiers)).astype(int)

    # Track the prediction of each classifier
    for classifier, predicted_class_index in enumerate(predicted_class_indices.tolist()):
      multiclass_predictions[classifier] = self.class_combinations[classifier][predicted_class_index]

    # Choose the class with the most votes
    y_hat = np.bincount(multiclass_predictions).argmax()

    return y_hat, multiclass_predictions

  def train(self, train_xs, train_ys):

    # Initialisation
    num_examples = train_xs.shape[0]
    self.train_xs = train_xs
    self.alpha = np.zeros((self.num_classifiers, num_examples))

    # Compute Gram matrix, K
    gram_matrix = self.kernel_function(train_xs, train_xs)

    # Training loop
    running_mistakes = 0
    train_accuracy_list = []

    converged = False
    epoch = 0

    while (converged == False) and (epoch <= MAX_EPOCHS):

      running_mistakes = 0

      # Shuffle the training data
      shuffled_indices = [*range(num_examples)]
      np.random.shuffle(shuffled_indices)

      for example in shuffled_indices:
        x = train_xs[example]
        y = train_ys[example]
        
        # Predict (most votes)
        y_hat, multiclass_predictions = self.__predict(gram_matrix, example)

        # Check if prediction is incorrect
        if (y_hat != y):
          running_mistakes += 1

        # Update relevant (i.e. binary prediction including label y) incorrect classifiers;
        # skip if classifier is correct or does not make prediction including label y
        for index, (i, j) in enumerate(self.class_combinations_to_alpha_index.keys()):
          prediction = multiclass_predictions[index]

          if y == i and prediction != y: # If so, y is the negative class for an incorrect classifier 
            self.alpha[index, example] -= 1
          elif y == j and prediction != y: # If so, y is the positive class for an incorrect classifier
            self.alpha[index, example] += 1
        
      # Calculate training accuracy
      train_accuracy = (num_examples - running_mistakes) / float(num_examples)
      train_accuracy_list.append(train_accuracy)

      # Early stopping
      if epoch >= MIN_EPOCHS:
        if (np.mean(train_accuracy_list[-2:]) - np.mean(train_accuracy_list[-4:-2])) < 0.01:
          converged = True

          # Count number of non-zero alpha vector elements per class
          print(f"Number of non-zero alpha vector elements per class: {np.count_nonzero(self.alpha, axis=1)}")
      
      epoch += 1
                
    return train_accuracy_list

  def test(self, test_xs, test_ys):

    num_examples = test_xs.shape[0]

    # Compute kernel matrix, K
    K_matrix = self.kernel_function(self.train_xs, test_xs)

    running_mistakes = 0
    confusion_matrix = np.zeros((self.num_classes, self.num_classes))

    # Make a prediction for each example
    for example in range(num_examples):
      y = test_ys[example]

      # Predict (most votes)
      y_hat, _ = self.__predict(K_matrix, example)

      # Track mistakes
      if (y_hat != y):
        running_mistakes += 1
        confusion_matrix[y][y_hat] += 1
    
    # Calculate test accuracy
    test_accuracy = (num_examples - running_mistakes) / float(num_examples)
    
    return test_accuracy, confusion_matrix

### 6.a. Basic results

In [ ]:
NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

DEGREES = range(1, 8)
RUNS = 20

summary_data = []

for degree in DEGREES:

  train_accuracy_list = np.empty((RUNS))
  test_accuracy_list = np.empty((RUNS))

  for run in range(RUNS):

    # Generate train/test split
    train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

    # Initialise model
    model = MultiClassPerceptronOvO(NUM_CLASSES)
    model.set_polynomial_kernel(degree)

    # Train model and evaluate on test set
    train_accuracy = model.train(train_xs, train_ys)[-1]
    test_accuracy, _ = model.test(test_xs, test_ys)

    train_accuracy_list[run] = train_accuracy
    test_accuracy_list[run] = test_accuracy

    print(f"Polynomial degree: {degree}. Run: {run}. Train accuracy: {train_accuracy}. Test accuracy: {test_accuracy}.")

  # Calculate mean and std for train and test accuracy
  train_error_mean, train_error_std = 1 - np.mean(train_accuracy_list), np.std(train_accuracy_list)
  test_error_mean, test_error_std = 1 - np.mean(test_accuracy_list), np.std(test_accuracy_list)

  summary_data.append([degree, train_error_mean, train_error_std, test_error_mean, test_error_std])
  
# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('degree', 'train_error_mean', 'train_error_std', 'test_error_mean', 'test_error_std'))

# Save summary as CSV
csv_filename = f"Q1_6_a_{datetime.now().strftime('%Y_%m_%d_%H_%M')}.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

### 6.b. Cross-validation

In [ ]:
NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

DEGREES = range(1, 8)
RUNS = 20

FOLDS = 5

summary_data = []
confusion_matrices = np.empty((RUNS, NUM_CLASSES, NUM_CLASSES))

for run in range(RUNS):

  mean_val_accuracy_list = np.empty((len(DEGREES)))

  # Generate train/test split
  train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

  for i, degree in enumerate(DEGREES):

    # Generate validation folds
    validation_folds = generate_validation_folds(train_xs, train_ys, k=5)

    train_accuracy_list = np.empty((FOLDS))
    val_accuracy_list = np.empty((FOLDS))

    for fold, (fold_train_xs, fold_train_ys, fold_validation_xs, fold_validation_ys) in enumerate(validation_folds):

      # Initialise model
      model = MultiClassPerceptronOvO(NUM_CLASSES)
      model.set_polynomial_kernel(degree=degree)

      # Train model and evaluate on validation set
      train_accuracy = model.train(fold_train_xs, fold_train_ys)[-1]
      val_accuracy, _ = model.test(fold_validation_xs, fold_validation_ys)

      train_accuracy_list[fold] = train_accuracy
      val_accuracy_list[fold] = val_accuracy

    print(f"Polynomial degree: {degree}. Run: {run}. Train accuracy list: {train_accuracy_list}. Val accuracy: {val_accuracy_list}.")

    mean_val_accuracy_list[i] = np.mean(val_accuracy_list)

  # Get the degree with maximum mean validation accuracy
  argmax = np.argmax(mean_val_accuracy_list)
  degree_star = DEGREES[argmax]
  
  # Initialise model
  model = MultiClassPerceptronOvO(NUM_CLASSES)
  model.set_polynomial_kernel(degree=degree)

  # Train model and evaluate on test set
  train_accuracy = model.train(train_xs, train_ys)[-1]
  test_accuracy, _ = model.test(test_xs, test_ys)

  train_error = 1 - train_accuracy
  test_error = 1 - test_accuracy

  summary_data.append([degree_star, train_error, test_error])
  print(summary_data)

# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('degree', 'train_error', 'test_error'))

# Save summary as CSV
csv_filename = f"Q1_6_b_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False)

summary

## 7. Comparison algorithms

### 7.a. Kernel SVM with CVXOPT

In [ ]:
class BinarySVM:
  """
  Binary kernel support vector machine (SVM)
  """
  def __init__(self, C = 1.0):
    """
    Args:
      C: The amount of regularization
    """

    self.C = C

    # Initialised by set_{polynomial/gaussian}_kernel method
    self.kernel_function = None

    # Initialised by train method
    self.alpha = None
    self.support_vectors = None
    self.support_labels = None
    self.bias = None
  
  def set_polynomial_kernel(self, degree):
    self.kernel_function = lambda x, y: polynomial_kernel(x, y, degree)
  
  def set_gaussian_kernel(self, width):
    self.kernel_function = lambda p, q: gaussian_kernel(p, q, width)

  def train(self, train_xs, train_ys):

    start = time.time()

    # Initialisation
    num_examples = train_xs.shape[0]

    # Compute Gram matrix, K
    K = self.kernel_function(train_xs, train_xs)

    # Formulate quadratic programming problem

    # (1/2) x.T P x + q.T x
    P = matrix(np.outer(train_ys, train_ys) * K)
    q = matrix(np.repeat(-1., num_examples))
    # Ax = b
    A = matrix(train_ys.astype(np.double), (1, num_examples))
    b = matrix(0.)
    # Gx <= h
    G = matrix(np.vstack((-np.eye(num_examples),
                           np.eye(num_examples))))
    h = matrix(np.hstack((np.zeros(num_examples),
                          np.ones(num_examples) * self.C)))
    
    print(f"Formulated {time.time() - start}")

    # Solve
    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)

    print(f"Solved {time.time() - start}")

    # Multipliers
    multipliers = np.ravel(solution['x'])
    
    # Support
    is_support_vector = multipliers > 1e-5
    support_vector_indices = np.flatnonzero(is_support_vector)
    
    self.alpha = multipliers[support_vector_indices]
    self.support_vectors = train_xs[support_vector_indices]
    self.support_labels = train_ys[support_vector_indices]

    # Bias
    self.bias = 0
    for i in range(len(self.alpha)):
      self.bias += self.support_labels[i]
      self.bias -= np.sum(self.alpha * self.support_labels * K[support_vector_indices[i], is_support_vector])
    self.bias /= len(self.alpha)

    return self

  def project(self, test_xs, test_ys):

    # Compute K matrix
    K = self.kernel_function(self.support_vectors, test_xs)

    num_examples = test_xs.shape[0]

    projections = np.empty((num_examples))

    for example in range(num_examples):
      projections[example] = self.bias + np.sum(self.alpha * self.support_labels * K[:, example])

    return projections
  
  def test(self, test_xs, test_ys):
    
    predictions = np.sign(self.project(test_xs, test_ys))

    return predictions

In [ ]:
class MulticlassSVM:
  """
  One-versus-rest (OvR) kernel support vector machine (SVM)
  """
  def __init__(self, num_classes, C = 1.0):
    """
    Args:
      num_classes: The number of classes
      C: The amount of regularization
    """

    self.num_classes = num_classes
    self.classifiers = [BinarySVM(C) for i in range(num_classes)]

  def set_polynomial_kernel(self, degree):
    for cfr in self.classifiers:
      cfr.set_polynomial_kernel(degree)
  
  def set_gaussian_kernel(self, width):
    for cfr in self.classifiers:
      cfr.set_gaussian_kernel(width)
  
  def train(self, train_xs, train_ys):

    # Training loop
    for i in range(self.num_classes):
      
      # Uncomment to use all training examples:
      # train_ys_i = np.where(train_ys == i, 1, -1)
      # self.classifiers[i].train(train_xs, train_ys_i)

      # Uncomment to use subsampling method:
      # Uses all training examples in class i, and a randomly selected set of
      # class non-i of the same size

      # Adjust labels to +1 for class i and -1 otherwise
      train_ys_i = np.where(train_ys == i, 1, -1)

      # Get indices of class i examples
      class_i_indices = np.where(train_ys_i == 1)
      class_i_num_examples = len(class_i_indices[0])

      # Get class_i_num_examples indices of class non-i examples at random
      non_class_i_indices = np.random.choice(np.where(train_ys_i == -1)[0], class_i_num_examples, replace=False)

      # Combine the i and non-i classes and randomly shuffle
      subset_indices = np.append(class_i_indices, non_class_i_indices)
      np.random.shuffle(subset_indices)

      train_xs_subset = train_xs[subset_indices]
      train_ys_subset = train_ys_i[subset_indices]

      self.classifiers[i].train(train_xs_subset, train_ys_subset)

    support_vectors_per_class = list(map(lambda clf: np.count_nonzero(clf.alpha), self.classifiers))
    print(f"Number of support vectors per class: {support_vectors_per_class}")

    return self

  def test(self, test_xs, test_ys):
    
    num_examples = test_xs.shape[0]

    projections = np.empty((self.num_classes, num_examples))

    # Compute projections for each classifier
    for i in range(self.num_classes):
      cfr = self.classifiers[i]
      projections[i] = cfr.project(test_xs, test_ys)
    
    # Predict the classes with the largest projections
    predictions = np.argmax(projections, axis=0)

    # Compute test accuracy
    test_accuracy = sum(predictions == test_ys) / num_examples

    return test_accuracy

In [ ]:
# 1. Basic results

NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

C_VALUES = [2 ** k for k in range(-1, 4)]
WIDTHS = [2 ** k for k in range(-7, -4)]

HYPERPARAMETERS = [(C, width) for C in C_VALUES for width in WIDTHS]

RUNS = 20

summary_data = []

for (C, width) in HYPERPARAMETERS:

  train_accuracy_list = np.empty((RUNS))
  test_accuracy_list = np.empty((RUNS))

  for run in range(RUNS):

    # Generate train/test split
    train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

    # Initialise model
    model = MulticlassSVM(num_classes = 10, C = C)
    model.set_gaussian_kernel(width)

    # Train model
    model.train(train_xs, train_ys)

    # Evaluate on training and test sets
    train_accuracy = model.test(train_xs, train_ys)
    test_accuracy = model.test(test_xs, test_ys)

    train_accuracy_list[run] = train_accuracy
    test_accuracy_list[run] = test_accuracy

    print(f"C: {C}. Gaussian kernel width: {width}. Run: {run}. Train accuracy: {train_accuracy}. Test accuracy: {test_accuracy}.")

  # Calculate mean and std for train and test accuracy
  train_error_mean, train_error_std = 1 - np.mean(train_accuracy_list), np.std(train_accuracy_list)
  test_error_mean, test_error_std = 1 - np.mean(test_accuracy_list), np.std(test_accuracy_list)

  summary_data.append([C, width, train_error_mean, train_error_std, test_error_mean, test_error_std])
  
# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('C', 'width', 'train_error_mean', 'train_error_std', 'test_error_mean', 'test_error_std'))

# Save summary as CSV
csv_filename = f"Q1_7a_1_{datetime.now().strftime('%Y_%m_%d_%H_%M')}.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

In [ ]:
# 2. Cross-validation

NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

C_VALUES = [2 ** k for k in range(-1, 4)]
WIDTHS = [2 ** k for k in range(-7, -4)]

HYPERPARAMETERS = [(C, width) for C in C_VALUES for width in WIDTHS]

RUNS = 20

FOLDS = 5

summary_data = []

for run in range(RUNS):

  mean_val_accuracy_list = np.empty((len(HYPERPARAMETERS)))

  # Generate train/test split
  train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

  for i, (C, width) in enumerate(HYPERPARAMETERS):

    # Generate validation folds
    validation_folds = generate_validation_folds(train_xs, train_ys, k=5)

    train_accuracy_list = np.empty((FOLDS))
    val_accuracy_list = np.empty((FOLDS))

    for fold, (fold_train_xs, fold_train_ys, fold_validation_xs, fold_validation_ys) in enumerate(validation_folds):

      # Initialise model
      model = MulticlassSVM(NUM_CLASSES, C=C)
      model.set_gaussian_kernel(width=width)

      # Train model
      model.train(fold_train_xs, fold_train_ys)

      # Evaluate on training and validation sets
      train_accuracy = model.test(fold_train_xs, fold_train_ys)
      val_accuracy = model.test(fold_validation_xs, fold_validation_ys)

      train_accuracy_list[fold] = train_accuracy
      val_accuracy_list[fold] = val_accuracy

    print(f"C: {C}. Gaussian kernel width: {width}. Run: {run}. Train accuracy list: {train_accuracy_list}. Val accuracy: {val_accuracy_list}.")

    mean_val_accuracy_list[i] = np.mean(val_accuracy_list)

  # Get the width with maximum mean validation accuracy
  argmax = np.argmax(mean_val_accuracy_list)
  (C_star, width_star) = HYPERPARAMETERS[argmax]
  
  # Initialise model
  model = MulticlassSVM(NUM_CLASSES, C=C_star)
  model.set_gaussian_kernel(width=width_star)

  # Train model
  model.train(train_xs, train_ys)

  # Evaluate on training and test sets
  train_accuracy = model.test(train_xs, train_ys)
  test_accuracy = model.test(test_xs, test_ys)

  train_error = 1 - train_accuracy
  test_error = 1 - test_accuracy

  summary_data.append([C_star, width_star, train_error, test_error])

# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('C', 'width', 'train_error', 'test_error'))

# Save summary as CSV
csv_filename = f"Q1_7a_2_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

### 7.a. Kernel SVM (one-versus-rest) with SMO solver

In [ ]:
class BinarySVM_SMO:
  """
  Binary kernel support vector machine (SVM) using sequential minimal optimization
  """
  def __init__(self, K, C = 1.0, tol=1e-5, max_iters=100):
    """
    Args:
      K: The Gram matrix for the training examples
      C: The amount of regularization
      tol: Numerical tolerance for Lagrange multipliers
      max_iters = The maximum number of passes over the Lagrange multipliers
    """

    self.K = K
    self.C = C
    self.tol = tol
    self.max_iters = max_iters
    self.max_iters_no_updates = 2

    self.num_examples = train_xs.shape[0]

    # Initialise
    self.alpha = np.zeros((self.num_examples))
    self.bias = 0

    # Initialised by set_gaussian_kernel
    self.kernel_function = None

    # Initialised by train method
    self.support_vectors = None
    self.support_labels = None
    self.train_xs = None
    self.train_ys = None

  def set_gaussian_kernel(self, width):
    self.kernel_function = lambda p, q: gaussian_kernel(p, q, width)

  def __calculate_E(self, index):
    """
    Calculate f(x_i) - y_i
    """
    f_x_index = np.dot((self.alpha * self.train_ys).T, self.K[:, index]) + self.bias
    return f_x_index - self.train_ys[index]

  def __randint_neq(self, start, stop, i):
    """
    Generate a random number between start and stop, that is not equal to i
    """
    j = randint(start, stop)
    while i == j:
      j = randint(start, stop)
    return j

  def train(self, train_xs, train_ys):

    self.train_xs = train_xs
    self.train_ys = train_ys

    # Track the number of consecutive iterations without updates, and the total number
    # of iterations
    iters_no_updates = 0
    iters = 0

    # Loop until no updates max_iters_no_updates times in a row, or reaches max_iters
    while(iters_no_updates < self.max_iters_no_updates and iters < self.max_iters):
      
      num_changed_alphas = 0
      
      for i in range(self.num_examples):
        
        y_i = self.train_ys[i]
        E_i = self.__calculate_E(i)

        if (y_i * E_i < -self.tol and self.alpha[i] < self.C) or \
          (y_i * E_i > self.tol and self.alpha[i] > 0):

          # Select random j != i
          j = self.__randint_neq(0, self.num_examples - 1, i)

          y_j = self.train_ys[j]
          E_j = self.__calculate_E(j)

          # Save old alphas
          alpha_i_old, alpha_j_old = self.alpha[i], self.alpha[j]

          if y_i != y_j:
            L = max(0, self.alpha[j] - self.alpha[i])
            H = min(self.C, self.C + self.alpha[j] - self.alpha[i])
          else:
            L = max(0, self.alpha[i] + self.alpha[j] - self.C)
            H = min(C, self.alpha[i] + self.alpha[j])

          if L == H:
            continue # next i
          
          eta = 2 * self.K[i, j] - self.K[i, i] - self.K[j ,j]

          if (eta >= 0):
            continue # next i
          
          # Clip alpha_j
          self.alpha[j] = alpha_j_old - (y_j*(E_i - E_j))/eta
          self.alpha[j] = min(self.alpha[j], H)
          self.alpha[j] = max(self.alpha[j], L)

          if abs(self.alpha[j] - alpha_j_old) < self.tol:
            continue # next i
          
          # Solve for alpha_i
          self.alpha[i] = alpha_i_old + y_i * y_j * (alpha_j_old - self.alpha[j])

          # Compute the bias
          b_1 = self.bias - E_i - y_i * (self.alpha[i] - alpha_i_old) * self.K[i, i] - y_j * (self.alpha[j] - alpha_j_old) * self.K[i, j]
          b_2 = self.bias - E_j - y_i * (self.alpha[i] - alpha_i_old) * self.K[i, j] - y_j * (self.alpha[j] - alpha_j_old) * self.K[j, j]

          if 0 < self.alpha[i] < self.C:
            self.bias = b_1 
          if 0 < self.alpha[j] < self.C:
            self.bias = b_2
          else:
            self.bias = (b_1 + b_2) / 2
          
          num_changed_alphas += 1
      
      iters += 1

      if num_changed_alphas == 0:
        iters_no_updates += 1
      else:
        iters_no_updates = 0

    # Support
    is_support_vector = self.alpha > 0
    support_vector_indices = np.flatnonzero(is_support_vector)
      
    self.alpha = self.alpha[support_vector_indices]
    self.support_vectors = train_xs[support_vector_indices]
    self.support_labels = train_ys[support_vector_indices]

  def project(self, test_xs, test_ys):

    # Compute K matrix
    K = self.kernel_function(self.support_vectors, test_xs)

    num_examples = test_xs.shape[0]

    projections = np.empty((num_examples))

    # Project each example
    for example in range(num_examples):
      projections[example] = self.bias + np.sum(self.alpha * self.support_labels * K[:, example])

    return projections
  
  def test(self, test_xs, test_ys):
    
    predictions = np.sign(self.project(test_xs, test_ys))

    return predictions

In [ ]:
class MulticlassSVM_SMO:
  """
  One-versus-rest (OvR) Gaussian kernel support vector machine (SVM) using
  sequential minimal optimization
  """
  def __init__(self, train_xs, train_ys, width, num_classes, C = 1.0):
    """
    Args:
      train_xs: The training examples
      train_ys: The training labels
      width: The Gaussian kernel width
      num_classes: The number of classes
      C: The amount of regularization
    """

    self.num_classes = num_classes

    # Compute Gram matrix, K
    K = gaussian_kernel(train_xs, train_xs, width)

    self.classifiers = [BinarySVM_SMO(K, C) for i in range(num_classes)]

    for cfr in self.classifiers:
      cfr.set_gaussian_kernel(width)
  
  def train(self):

    # Training loop
    for i in range(self.num_classes):

      print("Training:", i)
      train_ys_i = np.where(train_ys == i, 1, -1)
      self.classifiers[i].train(train_xs, train_ys_i)

    support_vectors_per_class = list(map(lambda clf: np.count_nonzero(clf.alpha), self.classifiers))
    print(f"Number of support vectors per class: {support_vectors_per_class}")

    return self

  def test(self, test_xs, test_ys):
    
    num_examples = test_xs.shape[0]

    projections = np.empty((self.num_classes, num_examples))

    # Compute projections for each classifier
    for i in range(self.num_classes):
      cfr = self.classifiers[i]
      projections[i] = cfr.project(test_xs, test_ys)
    
    # Predict the classes with the largest projections
    predictions = np.argmax(projections, axis=0)

    # Compute test accuracy
    test_accuracy = sum(predictions == test_ys) / num_examples

    return test_accuracy

In [ ]:
# 1. Basic results

NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

C_VALUES = [2 ** k for k in range(-1, 4)]
WIDTHS = [2 ** k for k in range(-7, -4)]

HYPERPARAMETERS = [(C, width) for C in C_VALUES for width in WIDTHS]

RUNS = 20

summary_data = []

for (C, width) in HYPERPARAMETERS:

  train_accuracy_list = np.empty((RUNS))
  test_accuracy_list = np.empty((RUNS))

  for run in range(RUNS):

    # Generate train/test split
    train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

    # Initialise model
    model = MulticlassSVM_SMO(train_xs, train_ys, width, num_classes = 10, C = C)

    # Train model
    model.train()

    # Evaluate on training and test sets
    train_accuracy = model.test(train_xs, train_ys)
    test_accuracy = model.test(test_xs, test_ys)

    train_accuracy_list[run] = train_accuracy
    test_accuracy_list[run] = test_accuracy

    print(f"C: {C}. Gaussian kernel width: {width}. Run: {run}. Train accuracy: {train_accuracy}. Test accuracy: {test_accuracy}.")

  # Calculate mean and std for train and test accuracy
  train_error_mean, train_error_std = 1 - np.mean(train_accuracy_list), np.std(train_accuracy_list)
  test_error_mean, test_error_std = 1 - np.mean(test_accuracy_list), np.std(test_accuracy_list)

  summary_data.append([C, width, train_error_mean, train_error_std, test_error_mean, test_error_std])
  
# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('C', 'width', 'train_error_mean', 'train_error_std', 'test_error_mean', 'test_error_std'))

# Save summary as CSV
csv_filename = f"Q1_7a_smo_1_{datetime.now().strftime('%Y_%m_%d_%H_%M')}.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

In [ ]:
# 2. Cross-validation

NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

C_VALUES = [2 ** k for k in range(-1, 4)]
WIDTHS = [2 ** k for k in range(-7, -4)]

HYPERPARAMETERS = [(C, width) for C in C_VALUES for width in WIDTHS]

RUNS = 20

FOLDS = 5

summary_data = []

for run in range(RUNS):

  mean_val_accuracy_list = np.empty((len(HYPERPARAMETERS)))

  # Generate train/test split
  train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

  for i, (C, width) in enumerate(HYPERPARAMETERS):

    # Generate validation folds
    validation_folds = generate_validation_folds(train_xs, train_ys, k=5)

    train_accuracy_list = np.empty((FOLDS))
    val_accuracy_list = np.empty((FOLDS))

    for fold, (fold_train_xs, fold_train_ys, fold_validation_xs, fold_validation_ys) in enumerate(validation_folds):

      # Initialise model
      model = MulticlassSVM_SMO(train_xs, train_ys, width, num_classes = 10, C = C)

      # Train model
      model.train()

      # Evaluate on training and validation sets
      train_accuracy = model.test(fold_train_xs, fold_train_ys)
      val_accuracy = model.test(fold_validation_xs, fold_validation_ys)

      train_accuracy_list[fold] = train_accuracy
      val_accuracy_list[fold] = val_accuracy

    print(f"C: {C}. Gaussian kernel width: {width}. Run: {run}. Train accuracy list: {train_accuracy_list}. Val accuracy: {val_accuracy_list}.")

    mean_val_accuracy_list[i] = np.mean(val_accuracy_list)

  # Get the width with maximum mean validation accuracy
  argmax = np.argmax(mean_val_accuracy_list)
  (C_star, width_star) = HYPERPARAMETERS[argmax]
  
  # Initialise model
  model = MulticlassSVM_SMO(train_xs, train_ys, width_star, num_classes = 10, C = C_star)

  # Train model
  model.train()

  # Evaluate on training and test sets
  train_accuracy = model.test(train_xs, train_ys)
  test_accuracy = model.test(test_xs, test_ys)

  train_error = 1 - train_accuracy
  test_error = 1 - test_accuracy

  summary_data.append([C_star, width_star, train_error, test_error])

# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('C', 'width', 'train_error', 'test_error'))

# Save summary as CSV
csv_filename = f"Q1_7a_smo_2_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

In [ ]:
from sklearn.tree import DecisionTreeClassifier

class MulticlassAdaBoost:
  """
  Multi-class AdaBoost (SAMME) classifier with decision trees as weak learners,
  using Gini impurity as the underlying metric
  """
  def __init__(self, num_classes, num_classifiers, max_depth):
    """
    Args:
      num_classes: The number of classes
      num_classifiers: The number of weak learners to train
      max_depth: The maximum depth of the decision trees
    """

    self.num_classes = num_classes
    self.num_classifiers = num_classifiers
    self.max_depth = max_depth

    # Initialised by train method
    self.classifiers = None
    self.learner_weights = None
  
  def train(self, train_xs, train_ys):

    num_examples, num_features = train_xs.shape

    # Initialise weights to 1/n
    example_weights = np.repeat(1/num_examples, num_examples)

    # Initialise weak learners and their weights
    self.classifiers = []
    self.learner_weights = np.empty((self.num_classifiers))

    for m in range(self.num_classifiers):

      # Train learner and predict on training set
      classifier = DecisionTreeClassifier(max_depth=self.max_depth)
      classifier.fit(train_xs, train_ys, sample_weight = example_weights)
      predictions = classifier.predict(train_xs)

      mistakes = predictions != train_ys

      # Compute the weighted error
      weighted_error = example_weights.dot(mistakes)

      # Compute the learner weight
      learner_weight = np.log(1 - weighted_error) - np.log(weighted_error) + np.log(self.num_classes - 1)

      # Scale and normalise weights
      example_weights *= np.exp(np.dot(learner_weight, mistakes))
      example_weights /= np.sum(example_weights)

      self.classifiers.append(classifier)
      self.learner_weights[m] = learner_weight

    return self

  def test(self, test_xs, test_ys):

      num_examples = test_xs.shape[0]
      labels = np.arange(self.num_classes)[:, np.newaxis]

      projections = np.zeros((num_examples, self.num_classes))

      for classifier, learner_weight in zip(self.classifiers, self.learner_weights):
        projections += np.dot(learner_weight, (classifier.predict(test_xs) == labels).T)

      predictions = np.argmax(projections, axis=1)

      # Compute test accuracy
      correct = predictions == test_ys
      test_accuracy = sum(correct) / num_examples

      return test_accuracy

In [ ]:
# 1. Basic results

NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

NUM_CLASSIFIERS = [25, 50, 100]
MAX_DEPTHS = [1, 2, 4, 8]

HYPERPARAMETERS = [(num_classifiers, max_depth) for num_classifiers in NUM_CLASSIFIERS for max_depth in MAX_DEPTHS]

RUNS = 20

summary_data = []

for (num_classifiers, max_depth) in HYPERPARAMETERS:

  train_accuracy_list = np.empty((RUNS))
  test_accuracy_list = np.empty((RUNS))

  for run in range(RUNS):

    # Generate train/test split
    train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

    # Initialise model
    model = MulticlassAdaBoost(NUM_CLASSES, num_classifiers, max_depth)

    # Train model
    model.train(train_xs, train_ys)

    # Evaluate on training and test sets
    train_accuracy = model.test(train_xs, train_ys)
    test_accuracy = model.test(test_xs, test_ys)

    train_accuracy_list[run] = train_accuracy
    test_accuracy_list[run] = test_accuracy

    print(f"Number of classifiers: {num_classifiers}. Max depth: {max_depth}. Train accuracy: {train_accuracy}. Test accuracy: {test_accuracy}.")

  # Calculate mean and std for train and test accuracy
  train_error_mean, train_error_std = 1 - np.mean(train_accuracy_list), np.std(train_accuracy_list)
  test_error_mean, test_error_std = 1 - np.mean(test_accuracy_list), np.std(test_accuracy_list)

  summary_data.append([num_classifiers, max_depth, train_error_mean, train_error_std, test_error_mean, test_error_std])
  
# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('num_classifiers', 'max_depth', 'train_error_mean', 'train_error_std', 'test_error_mean', 'test_error_std'))

# Save summary as CSV
csv_filename = f"Q1_7b_1_{datetime.now().strftime('%Y_%m_%d_%H_%M')}.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

In [ ]:
# 2. Cross-validation

NUM_CLASSES = 10

TRAIN_TEST_SPLIT = 0.8

NUM_CLASSIFIERS = [25, 50, 100]
MAX_DEPTHS = [1, 2, 4, 8]

HYPERPARAMETERS = [(num_classifiers, max_depth) for num_classifiers in NUM_CLASSIFIERS for max_depth in MAX_DEPTHS]

RUNS = 20

FOLDS = 5

summary_data = []

for run in range(RUNS):

  mean_val_accuracy_list = np.empty((len(HYPERPARAMETERS)))

  # Generate train/test split
  train_xs, train_ys, test_xs, test_ys = split_data(data_xs, data_ys, frac=TRAIN_TEST_SPLIT)

  for i, (num_classifiers, max_depth) in enumerate(HYPERPARAMETERS):

    # Generate validation folds
    validation_folds = generate_validation_folds(train_xs, train_ys, k=5)

    train_accuracy_list = np.empty((FOLDS))
    val_accuracy_list = np.empty((FOLDS))

    for fold, (fold_train_xs, fold_train_ys, fold_validation_xs, fold_validation_ys) in enumerate(validation_folds):

      # Initialise model
      model = MulticlassAdaBoost(NUM_CLASSES, num_classifiers, max_depth)

      # Train model
      model.train(fold_train_xs, fold_train_ys)

      # Evaluate on training and validation sets
      train_accuracy = model.test(fold_train_xs, fold_train_ys)
      val_accuracy = model.test(fold_validation_xs, fold_validation_ys)

      train_accuracy_list[fold] = train_accuracy
      val_accuracy_list[fold] = val_accuracy

    print(f"Number of classifiers: {num_classifiers}. Max depth: {max_depth}. Run: {run}. Train accuracy list: {train_accuracy_list}. Val accuracy: {val_accuracy_list}.")

    mean_val_accuracy_list[i] = np.mean(val_accuracy_list)

  # Get the width with maximum mean validation accuracy
  argmax = np.argmax(mean_val_accuracy_list)
  (num_classifiers_star, max_depth_star) = HYPERPARAMETERS[argmax]
  
  # Initialise model
  model = MulticlassAdaBoost(NUM_CLASSES, num_classifiers_star, max_depth_star)

  # Train model
  model.train(train_xs, train_ys)

  # Evaluate on training and test sets
  train_accuracy = model.test(train_xs, train_ys)
  test_accuracy = model.test(test_xs, test_ys)

  train_error = 1 - train_accuracy
  test_error = 1 - test_accuracy

  summary_data.append([num_classifiers_star, max_depth_star, train_error, test_error])

# Create a DataFrame of the summary data
summary = pd.DataFrame(summary_data, columns = ('num_classifiers', 'max_depth', 'train_error', 'test_error'))

# Save summary as CSV
csv_filename = f"Q1_7b_2_{datetime.now().strftime('%Y_%m_%d_%H_%M')}_{RUNS}_runs.csv"
summary.to_csv(f"{path}/{csv_filename}", index=False, float_format='%.3g')

summary

# Part II

## 1. Sparse learning

### Algorithms

In [ ]:
def generate_sample(m, n, algorithm=None):
  """
  Generate m patterns uniformly at random from {-1, 1}^n,
  or {0, 1}^n if the algorithm is winnow
  """
  if algorithm == winnow:
    X = np.random.choice([0, 1], (m, n))
    y = X[:,0]
  else:
    X = np.random.choice([-1, 1], (m, n))
    y = X[:,0]
  return X, y

def perceptron(X_train, y_train, X_test):
  """
  Train a perceptron on train patterns X_train and labels y_train,
  and evaluate on a test dataset X_test
  """
  weights = np.zeros((X_train.shape[1]))

  mistakes = 0

  # Training loop
  for i in range(X_train.shape[0]):
    x_i, y_i = X_train[i], y_train[i]

    y_i_hat = np.sign(weights @ x_i)

    # Update
    if (y_i_hat*y_i <= 0):
      weights += y_i*x_i
      mistakes += 1

  # Test
  return np.sign(X_test @ weights)

def winnow(X_train, y_train, X_test):
  """
  Train a winnow classifier on train patterns X_train and labels y_train,
  and evaluate on a test dataset X_test
  """
  n = X_train.shape[1]

  weights = np.ones((X_train.shape[1]))

  mistakes = 0

  # Training loop
  for i in range(X_train.shape[0]):
    x_i, y_i = X_train[i], y_train[i]

    y_i_hat = 0 if weights @ x_i < n else 1

    # Update
    if (y_i_hat != y_i):
      weights *= np.float_power(2, ((y_i - y_i_hat) * x_i))
      mistakes += 1

  # Test
  return np.where(X_test @ weights < n, 0, 1)

def least_squares(X_train, y_train, X_test):
  """
  Train a least squares classifier on train patterns X_train and labels y_train,
  and evaluate on a test dataset X_test
  """
  # Training
  weights = np.linalg.pinv(X_train) @ y_train

  # Test
  return np.sign(X_test @ weights)

def one_nearest_neighbour(X_train, y_train, X_test):
  """
  Given a dataset of train patterns X_train and labels y_train, evaluate a
  one-nearest-neighbour classifier on a test dataset X_test
  """
  X_train = np.expand_dims(X_train, 2)
  X_test = np.swapaxes(np.expand_dims(X_test, 2), 0, 2)

  # Test
  nearest = np.argmin(np.count_nonzero(X_train != X_test, axis = 1), axis = 0)
  return y_train[nearest]

### 1.a.

In [ ]:
def estimate_generalization_error(m, n, algorithm, runs):

  # Collect the test error for each run
  test_errors = np.empty((runs))

  for run in range(runs):

    # Generate a train and test sample
    X_train, y_train = generate_sample(m, n, algorithm)
    X_test, y_test = generate_sample(TEST_SIZE, n, algorithm)

    # Train algorithm and predict on the test sample
    predictions = algorithm(X_train, y_train, X_test)
    mistakes = np.count_nonzero(predictions != y_test)

    # Calculate the test error
    test_error = mistakes / TEST_SIZE

    test_errors[run] = test_error
  
  # Estimate the generalization error by taking the mean over the runs
  generalization_error = np.mean(test_errors)

  return generalization_error

def sample_complexity(algorithm, dimensions, runs):
  """
  Estimate the number of samples (m) to obtain 10% generalization error versus
  each dimension (n). Loop for m = 1, incrementing m, until the estimated
  generalization error is less than 0.1, for each dimension n.
  """

  # Collect estimated number of samples needed for each dimension
  estimated_samples_by_dimension = np.empty((len(dimensions)))

  for dimension_index, n in enumerate(dimensions):

      # Initialise
      m = 1
      generalization_error = float('inf')

      # Loop until estimated generalization error <= 0.1
      while m < MAX_M:
        
        generalization_error = estimate_generalization_error(m, n, algorithm, runs)

        if generalization_error <= 0.1:
          print(f"Dimension: {n}. Estimated m: {m}.")
          estimated_samples_by_dimension[dimension_index] = m
          break

        m += 1
  
  return estimated_samples_by_dimension

def sample_complexity_binary_search(algorithm, dimensions, runs, lower, upper):
  """
  Estimate the number of samples (m) to obtain 10% generalization error versus
  each dimension (n). Perform binary search between L and R to find the smallest
  such m for each n.
  """

  # Collect estimated number of samples needed for each dimension
  estimated_samples_by_dimension = np.empty((len(dimensions)))

  for dimension_index, n in enumerate(dimensions):

    L = lower
    R = upper

    # Initialise
    best_m = float('inf')

    # Loop until L == R
    while L <= R:

      if L == R or L + 1 == R:
        L_error = estimate_generalization_error(L, n, algorithm, runs)
        best_m = L if L_error < 0.1 else R
        estimated_samples_by_dimension[dimension_index] = best_m
        print(f"Dimension: {n}. Estimated m: {best_m}.")
        break

      m = (L + R) // 2

      generalization_error = estimate_generalization_error(m, n, algorithm, runs)

      if generalization_error < 0.1:
        R = m

        if m < best_m:
          best_m = m
          best_generalization_error = generalization_error
          print('Current best:', best_m, best_generalization_error)
      
      if generalization_error > 0.1:
        L = m
  
  return estimated_samples_by_dimension

In [ ]:
def plot_sample_complexity(algorithm, dimensions, samples_by_dimension_min):
  plt.plot(dimensions, samples_by_dimension_min)
  plt.xlabel('n')
  plt.ylabel('m')
  plt.title(f"Estimated number of samples (m) to obtain 10%\ngeneralisation error versus dimension (n) for {algorithm.__name__}")
  plt.savefig(f"Q2_a_{algorithm.__name__}", dpi=300)
  plt.show()

def plot_linear_fit(algorithm, dimensions, estimated_samples_by_dimension):
  a, b = np.polyfit(dimensions, estimated_samples_by_dimension, 1)

  # Print equation
  a_round, b_round = round(a, 3), round(b, 3)
  print(f"Algorithm: {algorithm.__name__}. Equation: {b_round} + {a_round}n.")

  # Plot estimate and fit
  plt.plot(dimensions, estimated_samples_by_dimension, label='Estimate')
  plt.plot(dimensions, a*dimensions + b, label='Fit')
  plt.xlabel('n')
  plt.ylabel('m')
  plt.legend()
  plt.title(f"Linear fit of number of samples (m) to obtain 10%\n generalisation error versus dimension (n) for {algorithm.__name__}")
  plt.savefig(f"Q2_a_{algorithm.__name__}_linear_fit", dpi=300)
  plt.show()

def plot_log_fit(algorithm, dimensions, estimated_samples_by_dimension):
  a, b = np.polyfit(np.log(dimensions), estimated_samples_by_dimension, 1)

  # Print equation
  a_round, b_round = round(a, 3), round(b, 3)
  print(f"Algorithm: {algorithm.__name__}. Equation: {b_round} + {a_round}log(n).")

  # Plot estimate and fit
  plt.plot(dimensions, estimated_samples_by_dimension, label='Estimate')
  plt.plot(dimensions, a*np.log(dimensions) + b, label='Fit')
  plt.xlabel('n')
  plt.ylabel('m')
  plt.legend()
  plt.title(f"Log fit of number of samples (m) to obtain 10%\ngeneralisation error versus dimension (n) for {algorithm.__name__}")
  plt.savefig(f"Q2_a_{algorithm.__name__}_log_fit", dpi=300)
  plt.show()

def plot_exponential_fit(algorithm, dimensions, estimated_samples_by_dimension):
  a, b = np.polyfit(dimensions, np.log(estimated_samples_by_dimension), 1)
  
  # Print equation
  a_round, b_round = round(a, 3), round(b, 3)
  print(f"Algorithm: {algorithm.__name__}. Equation: {b_round}e^({a_round})n.")

  # Plot estimate and fit
  plt.plot(dimensions, estimated_samples_by_dimension, label='Estimate')
  plt.plot(dimensions, np.exp(a*dimensions + b), label='Fit')
  plt.xlabel('n')
  plt.ylabel('m')
  plt.legend()
  plt.title(f"Exponential fit of number of samples (m) to obtain 10%\ngeneralisation error versus dimension (n) for {algorithm.__name__}")
  plt.savefig(f"Q2_a_{algorithm.__name__}_exponential_fit", dpi=300)
  plt.show()

In [ ]:
# Constants
RUNS = 5
TEST_SIZE = 10000
MAX_M = 10000 # Max sample size

In [ ]:
# Perceptron
dimensions = range(1, 101)

estimated_samples_by_dimension = sample_complexity(perceptron, dimensions, RUNS)

plot_sample_complexity(perceptron, dimensions, estimated_samples_by_dimension)
plot_linear_fit(perceptron, dimensions, estimated_samples_by_dimension)

In [ ]:
# Winnow
dimensions = range(1, 101)

estimated_samples_by_dimension = sample_complexity(winnow, dimensions, RUNS)

plot_sample_complexity(winnow, dimensions, estimated_samples_by_dimension)
plot_log_fit(winnow, dimensions, estimated_samples_by_dimension)

In [ ]:
# Least squares
dimensions = range(100, 101)

estimated_samples_by_dimension = sample_complexity(least_squares, dimensions, RUNS)

plot_sample_complexity(least_squares, dimensions, estimated_samples_by_dimension)
plot_linear_fit(least_squares, dimensions, estimated_samples_by_dimension)

In [ ]:
# 1-nearest neighbours
dimensions = range(1, 21)

estimated_samples_by_dimension = sample_complexity(one_nearest_neighbour, dimensions, RUNS)

plot_sample_complexity(one_nearest_neighbour, dimensions, estimated_samples_by_dimension)
plot_exponential_fit(one_nearest_neighbour, dimensions, estimated_samples_by_dimension)